In [1]:
!pip install pandasql
import requests
import pandas as pd
import token
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=b932a22079429d8cde724d60976c4a8a44e1ca54ed779214e923460983253f02
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [2]:
# if you do not have write access to this path, you will need 
# to change it to a path that you do have write access to.
gdrivePath="/content/drive/MyDrive/"
import sqlite3
import pandas as pd
import os
import pandasql
try:
  os.remove(gdrivePath+"CS.db") # remove the database if it is already there
  print("removed earlier copy of CS.db")
except:
  print("not already there")  
# create a connection to a database
conn = sqlite3.connect(gdrivePath+"CS.db")

removed earlier copy of CS.db


In [3]:
import json
  
# Opening JSON file
f = open('/content/drive/MyDrive/bios.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
data

[{'_id': 1,
  'name': {'first': 'John', 'last': 'Backus'},
  'birth': '1924-12-03',
  'death': '2007-03-17',
  'contribs': ['Fortran', 'ALGOL', 'Backus-Naur Form', 'FP'],
  'awards': [{'award': 'W.W. McDowell Award',
    'year': 1967,
    'by': 'IEEE Computer Society'},
   {'award': 'National Medal of Science',
    'year': 1975,
    'by': 'National Science Foundation'},
   {'award': 'Turing Award', 'year': 1977, 'by': 'ACM'},
   {'award': 'Draper Prize',
    'year': 1993,
    'by': 'National Academy of Engineering'}]},
 {'_id': 2,
  'name': {'first': 'John', 'last': 'McCarthy'},
  'birth': '1927-09-04',
  'death': '2011-12-24',
  'contribs': ['Lisp', 'Artificial Intelligence', 'ALGOL'],
  'awards': [{'award': 'Turing Award', 'year': 1971, 'by': 'ACM'},
   {'award': 'Kyoto Prize', 'year': 1988, 'by': 'Inamori Foundation'},
   {'award': 'National Medal of Science',
    'year': 1990,
    'by': 'National Science Foundation'}]},
 {'_id': 3,
  'name': {'first': 'Grace', 'last': 'Hopper'},
  

In [4]:
df = pd.json_normalize(data)
df = df.rename(columns={'name.first': 'first_name', 'name.last':'last_name', 'name.aka':'aka'})
df

,_id,birth,death,contribs,awards,first_name,last_name,title,aka
0,1,1924-12-03,2007-03-17,"[Fortran, ALGOL, Backus-Naur Form, FP]","[{'award': 'W.W. McDowell Award', 'year': 1967...",John,Backus,NaN,NaN
1,2,1927-09-04,2011-12-24,"[Lisp, Artificial Intelligence, ALGOL]","[{'award': 'Turing Award', 'year': 1971, 'by':...",John,McCarthy,NaN,NaN
2,3,1906-12-09,1992-01-01,"[UNIVAC, compiler, FLOW-MATIC, COBOL]",[{'award': 'Computer Sciences Man of the Year'...,Grace,Hopper,Rear Admiral,NaN
3,4,1926-08-27,2002-08-10,"[OOP, Simula]","[{'award': 'Rosing Prize', 'year': 1999, 'by':...",Kristen,Nygaard,NaN,NaN
4,5,1931-10-12,2002-06-29,"[OOP, Simula]","[{'award': 'Rosing Prize', 'year': 1999, 'by':...",Ole-Johan,Dahl,NaN,NaN
5,6,1956-01-31,NaN,[Python],[{'award': 'Award for the Advancement of Free ...,Guido,van Rossum,NaN,NaN
6,7,1941-09-09,2011-10-12,"[UNIX, C]","[{'award': 'Turing Award', 'year': 1983, 'by':...",Dennis,Ritchie,NaN,NaN
7,8,1965-04-14,NaN,[Ruby],[{'award': 'Award for the Advancement of Free ...,Yukihiro,Matsumoto,NaN,Matz
8,9,1955-05-19,NaN,[Java],"[{'award': 'The Economist Innovation Award', '...",James,Gosling,NaN,NaN
9,10,NaN,NaN,[Scala],NaN,Martin,Odersky,NaN,NaN


In [5]:
scientist = df.drop(['contribs', 'awards', 'title'],axis=1)
scientist

,_id,birth,death,first_name,last_name,aka
0,1,1924-12-03,2007-03-17,John,Backus,NaN
1,2,1927-09-04,2011-12-24,John,McCarthy,NaN
2,3,1906-12-09,1992-01-01,Grace,Hopper,NaN
3,4,1926-08-27,2002-08-10,Kristen,Nygaard,NaN
4,5,1931-10-12,2002-06-29,Ole-Johan,Dahl,NaN
5,6,1956-01-31,NaN,Guido,van Rossum,NaN
6,7,1941-09-09,2011-10-12,Dennis,Ritchie,NaN
7,8,1965-04-14,NaN,Yukihiro,Matsumoto,Matz
8,9,1955-05-19,NaN,James,Gosling,NaN
9,10,NaN,NaN,Martin,Odersky,NaN


In [6]:
scientist.to_csv('/content/drive/MyDrive/computer_scientists.csv',index=False) 

In [7]:
cursor = conn.cursor()
 
# Table Definition
create_table = '''
CREATE TABLE "computer_scientists" (
	"_id"	INTEGER,
	"birth"	TEXT,
	"death"	TEXT,
	"first_name"	TEXT,
	"last_name"	TEXT,
	"preferred"	TEXT,
	PRIMARY KEY("_id")
);
'''
 
# Creating the table into our
# database
cursor.execute(create_table)
 
# Opening the person-records.csv file
file0 = open('/content/drive/MyDrive/computer_scientists.csv')
# Reading the contents of the
# person-records.csv file
contents = csv.reader(file0)
next(contents)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO computer_scientists(_id, birth, death, first_name, last_name, preferred) VALUES(?, ?, ?, ?, ?, ?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM computer_scientists"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)
 
# Committing the changes
conn.commit()

(1, '1924-12-03', '2007-03-17', 'John', 'Backus', '')
(2, '1927-09-04', '2011-12-24', 'John', 'McCarthy', '')
(3, '1906-12-09', '1992-01-01', 'Grace', 'Hopper', '')
(4, '1926-08-27', '2002-08-10', 'Kristen', 'Nygaard', '')
(5, '1931-10-12', '2002-06-29', 'Ole-Johan', 'Dahl', '')
(6, '1956-01-31', '', 'Guido', 'van Rossum', '')
(7, '1941-09-09', '2011-10-12', 'Dennis', 'Ritchie', '')
(8, '1965-04-14', '', 'Yukihiro', 'Matsumoto', 'Matz')
(9, '1955-05-19', '', 'James', 'Gosling', '')
(10, '', '', 'Martin', 'Odersky', '')


In [8]:
contribs = df.drop(['birth', 'death', 'first_name', 'last_name', 'aka', 'awards', 'title'],axis=1)
contribs = contribs.explode('contribs', ignore_index=True)
contribs

,_id,contribs
0,1,Fortran
1,1,ALGOL
2,1,Backus-Naur Form
3,1,FP
4,2,Lisp
5,2,Artificial Intelligence
6,2,ALGOL
7,3,UNIVAC
8,3,compiler
9,3,FLOW-MATIC


In [9]:
contribs.to_csv('/content/drive/MyDrive/contribs.csv',index=False) 

In [10]:
# Table Definition
create_table = '''
CREATE TABLE "contribs" (
	"_id"	INTEGER,
	"contribs"	TEXT,
	FOREIGN KEY("_id") REFERENCES "computer_scientists"("_id")
);
'''
 
# Creating the table into our
# database
cursor.execute(create_table)
 
# Opening the person-records.csv file
file0 = open('/content/drive/MyDrive/contribs.csv')
# Reading the contents of the
# person-records.csv file
contents = csv.reader(file0)
next(contents)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO contribs(_id, contribs) VALUES(?, ?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM contribs"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)
 
# Committing the changes
conn.commit()

(1, 'Fortran')
(1, 'ALGOL')
(1, 'Backus-Naur Form')
(1, 'FP')
(2, 'Lisp')
(2, 'Artificial Intelligence')
(2, 'ALGOL')
(3, 'UNIVAC')
(3, 'compiler')
(3, 'FLOW-MATIC')
(3, 'COBOL')
(4, 'OOP')
(4, 'Simula')
(5, 'OOP')
(5, 'Simula')
(6, 'Python')
(7, 'UNIX')
(7, 'C')
(8, 'Ruby')
(9, 'Java')
(10, 'Scala')


In [11]:
awards = df.drop(['birth', 'death', 'first_name', 'last_name', 'aka', 'contribs', 'title'],axis=1)
awards = awards.explode('awards', ignore_index=True)
awards = awards.join(pd.json_normalize(awards.pop('awards')))
awards

,_id,award,year,by
0,1,W.W. McDowell Award,1967,IEEE Computer Society
1,1,National Medal of Science,1975,National Science Foundation
2,1,Turing Award,1977,ACM
3,1,Draper Prize,1993,National Academy of Engineering
4,2,Turing Award,1971,ACM
5,2,Kyoto Prize,1988,Inamori Foundation
6,2,National Medal of Science,1990,National Science Foundation
7,3,Computer Sciences Man of the Year,1969,Data Processing Management Association
8,3,Distinguished Fellow,1973,British Computer Society
9,3,W. W. McDowell Award,1976,IEEE Computer Society


In [12]:
awards.to_csv('/content/drive/MyDrive/awards.csv',index=False) 

In [13]:
# Table Definition
create_table = '''
CREATE TABLE "awards" (
	"_id"	INTEGER,
	"award"	TEXT,
  "year" TEXT,
  "by" TEXT,
	FOREIGN KEY("_id") REFERENCES "computer_scientists"("_id")
);
'''
 
# Creating the table into our
# database
cursor.execute(create_table)
 
# Opening the person-records.csv file
file0 = open('/content/drive/MyDrive/awards.csv')
# Reading the contents of the
# person-records.csv file
contents = csv.reader(file0)
next(contents)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO awards(_id, award, year, by) VALUES(?, ?, ? ,?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM awards"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)
 
# Committing the changes
conn.commit()

(1, 'W.W. McDowell Award', '1967', 'IEEE Computer Society')
(1, 'National Medal of Science', '1975', 'National Science Foundation')
(1, 'Turing Award', '1977', 'ACM')
(1, 'Draper Prize', '1993', 'National Academy of Engineering')
(2, 'Turing Award', '1971', 'ACM')
(2, 'Kyoto Prize', '1988', 'Inamori Foundation')
(2, 'National Medal of Science', '1990', 'National Science Foundation')
(3, 'Computer Sciences Man of the Year', '1969', 'Data Processing Management Association')
(3, 'Distinguished Fellow', '1973', ' British Computer Society')
(3, 'W. W. McDowell Award', '1976', 'IEEE Computer Society')
(3, 'National Medal of Technology', '1991', 'United States')
(4, 'Rosing Prize', '1999', 'Norwegian Data Association')
(4, 'Turing Award', '2001', 'ACM')
(4, 'IEEE John von Neumann Medal', '2001', 'IEEE')
(5, 'Rosing Prize', '1999', 'Norwegian Data Association')
(5, 'Turing Award', '2001', 'ACM')
(5, 'IEEE John von Neumann Medal', '2001', 'IEEE')
(6, 'Award for the Advancement of Free Software'

In [14]:
title = df[['_id','title']]
title

,_id,title
0,1,NaN
1,2,NaN
2,3,Rear Admiral
3,4,NaN
4,5,NaN
5,6,NaN
6,7,NaN
7,8,NaN
8,9,NaN
9,10,NaN


In [15]:
title.to_csv('/content/drive/MyDrive/title.csv',index=False) 

In [16]:
# Table Definition
create_table = '''
CREATE TABLE "title" (
	"_id"	INTEGER,
	"title"	TEXT,
  PRIMARY KEY("_id")
	FOREIGN KEY("_id") REFERENCES "computer_scientists"("_id")
);
'''
 
# Creating the table into our
# database
cursor.execute(create_table)
 
# Opening the person-records.csv file
file0 = open('/content/drive/MyDrive/title.csv')
# Reading the contents of the
# person-records.csv file
contents = csv.reader(file0)
next(contents)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO title(_id, title) VALUES(?, ?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM title"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)
 
# Committing the changes
conn.commit()

(1, '')
(2, '')
(3, 'Rear Admiral')
(4, '')
(5, '')
(6, '')
(7, '')
(8, '')
(9, '')
(10, '')


In [17]:
date = df[['_id','birth','death']]
date

,_id,birth,death
0,1,1924-12-03,2007-03-17
1,2,1927-09-04,2011-12-24
2,3,1906-12-09,1992-01-01
3,4,1926-08-27,2002-08-10
4,5,1931-10-12,2002-06-29
5,6,1956-01-31,NaN
6,7,1941-09-09,2011-10-12
7,8,1965-04-14,NaN
8,9,1955-05-19,NaN
9,10,NaN,NaN


In [18]:
date.to_csv('/content/drive/MyDrive/date.csv',index=False) 

In [19]:
# Table Definition
create_table = '''
CREATE TABLE "date" (
	"_id"	INTEGER,
	"birth"	date,
  "death" date,
  PRIMARY KEY("_id")
	FOREIGN KEY("_id") REFERENCES "computer_scientists"("_id")
);
'''
 
# Creating the table into our
# database
cursor.execute(create_table)
 
# Opening the person-records.csv file
file0 = open('/content/drive/MyDrive/date.csv')
# Reading the contents of the
# person-records.csv file
contents = csv.reader(file0)
next(contents)
 
# SQL query to insert data into the
# person table
insert_records = "INSERT INTO date(_id, birth, death) VALUES(?, ?, ?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
 
# SQL query to retrieve all data from
# the person table To verify that the
# data of the csv file has been successfully
# inserted into the table
select_all = "SELECT * FROM date"
rows = cursor.execute(select_all).fetchall()
 
# Output to the console screen
for r in rows:
    print(r)
 
# Committing the changes
conn.commit()

(1, '1924-12-03', '2007-03-17')
(2, '1927-09-04', '2011-12-24')
(3, '1906-12-09', '1992-01-01')
(4, '1926-08-27', '2002-08-10')
(5, '1931-10-12', '2002-06-29')
(6, '1956-01-31', '')
(7, '1941-09-09', '2011-10-12')
(8, '1965-04-14', '')
(9, '1955-05-19', '')
(10, '', '')


In [20]:
def allrowsSelect(conn,query):
  cursor = conn.execute(query)
  for row in cursor:
   print(row)

In [22]:
query='''
SELECT ROUND(CAST(total_count as float)/num_artist, 2)  AS avg_awards
FROM (SELECT	
(SELECT count(award) FROM awards ) AS total_count,
(SELECT count (*)  FROM computer_scientists) AS num_artist);
'''
allrowsSelect(conn,query)

(2.6,)
